In [1]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim

In [ ]:
df = pd.read_csv("dados_simulacao/distancias_todas_combinacoes.csv")
df

,Estado_A,Filial_A,Estado_B,Filial_B,Distancia_km
0,RJ,CENTRO DE DISTRIBUICAO,SP,ANIMALE HIGIENOPOLIS CM,348.796
1,RJ,CENTRO DE DISTRIBUICAO,RJ,ANIMALE ICARAI 02 CM,26.246
2,RJ,CENTRO DE DISTRIBUICAO,RJ,ANIMALE ICARAI CM,26.386
3,RJ,CENTRO DE DISTRIBUICAO,SC,ANIMALE IG FLORIPA CM,742.973
4,RJ,CENTRO DE DISTRIBUICAO,SP,ANIMALE IG RIBEIRAO CM,495.541
...,...,...,...,...,...
1948,RS,ANIMALE MOINHOS CM,PR,ANIMALE MUELLER CM,544.883
1949,RS,ANIMALE MOINHOS CM,RN,ANIMALE NATAL CM,3158.836
1950,SP,ANIMALE MORUMBI CM,PR,ANIMALE MUELLER CM,327.779
1951,SP,ANIMALE MORUMBI CM,RN,ANIMALE NATAL CM,2320.905


In [17]:
# definição dos grupos pra facilitar a vida

todas_com_cd = df
todas_sem_cd = df[(df['Filial_A'] != 'CENTRO DE DISTRIBUICAO') 
                      & (df['Filial_B'] != 'CENTRO DE DISTRIBUICAO')]

rj_com_cd = df[(df['Estado_A'] == 'RJ')|(df['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ

rj_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'RJ')|(todas_sem_cd['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ \{filial = CD}

sp_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'SP')|(todas_sem_cd['Estado_B'] == 'SP')] # uniao de E_A e E_B == SP

cd_sp = df[((df['Estado_A'] == 'SP') & (df['Filial_B'] == 'CENTRO DE DISTRIBUICAO'))
           | ((df['Estado_B'] == 'SP') & (df['Filial_A'] == 'CENTRO DE DISTRIBUICAO'))]
sp_com_cd = pd.concat((cd_sp, sp_sem_cd), ignore_index=True)  # uniao de E_A e E_B == SP  +  {A = CD e E_B = SP}


estado_sem_cd = None 
estado_com_cd = None 




In [4]:
# Variáveis a serem alteradas 

path = todas_sem_cd
cd = False # booleano se considera ou não o CD no grafo
custo_km = 0.5  # custo do frete em reais por km

In [ ]:
# Criar o  grafo

